In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-oOk_HttAhCaYywgXnvol4KT3BlbkFJDU9Ym6K6XX9DausZXs6u5Ka9uloCloKkwdpi5rS8SuCdBFt_HWxyPR5sfXvp2I1Q0R0alskNkA"


In [ ]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# Step 1a- Indexing(Document Ingestion)

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "hEpBWippEZY"

try:
    # create an instance
    api = YouTubeTranscriptApi()

    # list transcripts for this video
    transcript_list = api.list(video_id)

    # pick English transcript and fetch it
    transcript = transcript_list.find_transcript(["en"]).fetch()

    # convert to dictionary format
    transcript = [
        {
            "text": chunk.text,
            "start_time": chunk.start,
            "end_time": chunk.start + chunk.duration
        }
        for chunk in transcript
    ]

    # print dictionary list
    for entry in transcript:
        print(entry)

except Exception as e:
    print(f"Error: {e}")


{'text': "Are you somebody who's struggling to get", 'start_time': 0.0, 'end_time': 3.2}
{'text': 'back to studies after a long break? Or', 'start_time': 1.439, 'end_time': 4.720000000000001}
{'text': "you're somebody who was studying very", 'start_time': 3.2, 'end_time': 6.640000000000001}
{'text': "well but somehow suddenly can't find the", 'start_time': 4.72, 'end_time': 8.8}
{'text': 'motivation to start studying back again.', 'start_time': 6.64, 'end_time': 10.48}
{'text': 'You might be preparing for that one exam', 'start_time': 8.8, 'end_time': 12.16}
{'text': 'which is really important or that one', 'start_time': 10.48, 'end_time': 14.08}
{'text': "test coming up next Sunday. Here's your", 'start_time': 12.16, 'end_time': 16.32}
{'text': 'complete guide on how to get out of a', 'start_time': 14.08, 'end_time': 18.72}
{'text': 'study slump if you are in one and also', 'start_time': 16.32, 'end_time': 20.48}
{'text': 'on how to increase your productivity', 'start_time': 18.72, 'e

# Step 1-b Indexing (Text Splitter)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# join transcript dicts into one long string
full_text = " ".join(entry["text"] for entry in transcript)

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([full_text])

print(chunks[0].page_content[:300])  # preview first chunk


Are you somebody who's struggling to get back to studies after a long break? Or you're somebody who was studying very well but somehow suddenly can't find the motivation to start studying back again. You might be preparing for that one exam which is really important or that one test coming up next S


In [ ]:
len(chunks)

25

In [ ]:
chunks[0]

Document(metadata={}, page_content="Are you somebody who's struggling to get back to studies after a long break? Or you're somebody who was studying very well but somehow suddenly can't find the motivation to start studying back again. You might be preparing for that one exam which is really important or that one test coming up next Sunday. Here's your complete guide on how to get out of a study slump if you are in one and also on how to increase your productivity even if you are not feeling like studying. Hello everyone, I'm Dr. Dr. Anush Pachel and I am a first year MD medicine resident at GMC Nagpur and in this video I'll be giving you a straightforward guide filled with 10 tips that you can apply to get out of a study slump instantly. I have been an MBBS student and now I'm an MD student and I've given hundreds of exam in my life and trust me I know the feeling of that study slump. So without any further ado let's start. Step one is identifying that if you are actually in a study s

# Step 1c & 1d -indexing (embedding generation and storing in vector store)

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embeddings)

In [ ]:
vector_store.index_to_docstore_id

{0: '2a40a9b1-6817-4dbc-80f3-d940b02bd1bd',
 1: 'b6d4969b-27af-4840-b4a0-95012e84c1c3',
 2: '73b29b55-af97-4470-9467-6851530fb60a',
 3: '4e35ae65-3491-4dd4-b302-3c407ba430e0',
 4: 'e8676bdd-2dad-41df-ba87-6857a6749adc',
 5: '70070968-b353-4b95-9389-26b617a25fbf',
 6: '1b79aeda-7fd7-4977-9daf-f70957d12c96',
 7: '1db5bfa4-3e8a-4a0d-b1fc-3995e7028785',
 8: '3a60e24a-0a30-4c3c-ab14-73b1e06d441c',
 9: '5b05b173-9a5d-4ee5-b17d-1b92a4ab1dda',
 10: '7743cc94-be3c-4a6b-819d-ade151f6e2ec',
 11: '5e263ed8-9401-42c1-a735-c33f93e00951',
 12: '783604cc-8a81-4239-b9d0-dea2eeed1a13',
 13: '3b0eadaf-ccf4-4258-ba87-57c786bcefea',
 14: '66a09b16-3b81-4913-a431-a73d6e60a8b5',
 15: '03d1130c-ce15-4a43-8f70-842ea7a0db25',
 16: '8a15db30-8cb5-47ac-b117-5b8322916723',
 17: '55e9d988-4900-4436-b002-f8d74cab098b',
 18: '2ab198e4-a15d-438d-8b29-56d6ce7b3f7d',
 19: 'cc9fd613-14c0-431b-ad14-76edbf3ebe74',
 20: 'a3d0d6ce-243e-438f-82f0-5fd0a250cbf0',
 21: 'a3d6412b-debe-4d99-8dd8-9033f0ba5b18',
 22: '88d3ad97-d55a-

In [ ]:
vector_store.get_by_ids(['55e9d988-4900-4436-b002-f8d74cab098b'])

[Document(id='55e9d988-4900-4436-b002-f8d74cab098b', metadata={}, page_content="big reason maybe it's your parents that that you want to see them happy. Maybe it's your sister that you promise to I will crack this exam no matter what. Maybe it's your mentor. motivates me that you promise to just remind yourself of why you started my friend. And for all the students who are preparing for competitive examinations, there is something I want to tell you. If you are not preparing with your 100%, if you're not giving all of your efforts, if you're not giving that one last drop of everything that you can give, you should not be preparing. That's the simple simple fun. If you're not giving 100%, don't give 99% efforts. Just stop giving efforts because even if you're giving 99% efforts, you are not going to get selected. That's the simple ideology. The number of seats are so low. 20 lakh people are appearing. One lakh seats are there. 5% selection ratio. If you are not in the top 10 people, the

# Step 2 - Retrieval

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
retriever.invoke('What is Motivation')

[Document(id='8a15db30-8cb5-47ac-b117-5b8322916723', metadata={}, page_content="I have taken to start studying in the first place and I cannot let this go. So my main goal was to I'll never take a drop and that's the way I studied consistently. I reminded myself yeah if I don't study I have to take a drop. So I reminded myself of why I'm actually studying. Okay I want to become a doctor that's always secondary but that was my main motivation in my need PG. My motivation was I need to get a rank so decent that I get a choice of my branch and I get a choice of my college. So there are so many branches that you can there are almost like 36 specialties that you can opt for after an EPG is done. So to get a branch of your choice you really have to have a good rank. So you have to study according to that. If you keep this motivation in mind if you feel like there is a certain big reason maybe it's your parents that that you want to see them happy. Maybe it's your sister that you promise to I

## step - 3 Augmentation

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say NO.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [ ]:
question          = "is the topic of motivation discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

[Document(id='73b29b55-af97-4470-9467-6851530fb60a', metadata={}, page_content="place, that's when you know that you have lost the motivation to study. So these are the few signs. If you have any of these then make sure that you follow this video carefully. So the immediate next step that you can do in your life is to change the environment in which you are studying. What I used to do is that I used to feel a study's lump whenever I was sitting in my room for long periods of time like months at a stretch. And I used to lose all the motivation to study. But once I joined the library that's when you find a real motivation to start studying back again. This was a serious case I had and finally I remember where I just would not study if I was in my room. So I had to sit in the library for an entirety of days in order to study well. So first step is changing the place of your study. If you're not too attached with the space of study, that is if you don't have all the notes around all the bo

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"place, that's when you know that you have lost the motivation to study. So these are the few signs. If you have any of these then make sure that you follow this video carefully. So the immediate next step that you can do in your life is to change the environment in which you are studying. What I used to do is that I used to feel a study's lump whenever I was sitting in my room for long periods of time like months at a stretch. And I used to lose all the motivation to study. But once I joined the library that's when you find a real motivation to start studying back again. This was a serious case I had and finally I remember where I just would not study if I was in my room. So I had to sit in the library for an entirety of days in order to study well. So first step is changing the place of your study. If you're not too attached with the space of study, that is if you don't have all the notes around all the books and you can easily change your place, go do that for at least a few days or

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say NO.\n\n      place, that's when you know that you have lost the motivation to study. So these are the few signs. If you have any of these then make sure that you follow this video carefully. So the immediate next step that you can do in your life is to change the environment in which you are studying. What I used to do is that I used to feel a study's lump whenever I was sitting in my room for long periods of time like months at a stretch. And I used to lose all the motivation to study. But once I joined the library that's when you find a real motivation to start studying back again. This was a serious case I had and finally I remember where I just would not study if I was in my room. So I had to sit in the library for an entirety of days in order to study well. So first step is changing the place of your study. If you're 

# Step - 4 Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)


Yes, the topic of motivation is discussed in this video. It addresses signs of losing motivation to study, such as feeling overwhelmed by the syllabus, procrastination, and an inability to focus. The speaker suggests changing the study environment, reminding oneself of the reasons for studying, and staying physically active to boost motivation. Additionally, it emphasizes the importance of using active study methods and techniques to help overcome a study slump.


## Building a Chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('what is energy')

{'context': "in the morning or in the night. Now my general conclusion, my general ideology for most of the people is that even though that you think that you are a night person, it's not like that. If you try to wake up for a few mornings, maybe just 1 hour before your normal time, you will see that mornings are so productive. Now I'll tell you the reason for this. See, in the morning when you wake up, you're full of energy because you've just woken up from your sleep. You have got all your rest. Your brain is fully active. Now you you have a huge energy store. Now, as the day progresses, there are certain activities that you do that take away your energy. So what our idea is that in our evening our energy stores are low. So whatever things we are studying with a low energy reserve, they're not going to be going in your brain that easily because it's low on energy. It's it's taking more energy um to learn new information to focus. Right? So in the morning when you have a lot of energy

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('is the topic of making money her , if yes tell me about it in 2 lines')

'NO'